<h1> How to save Gotham </h1>

<h3> Environment </h3>
Create a environment
<ul>
    <li> State Space</li>
    <li> Action space</li>
    <li> Reward functions</li>
    <li>Constraints</li>
    <li> Reset and step functions</li>
</ul>

<h3> Agent </h3>
Create one or mulitple agents to interact with the enivornment
<ul>
    <li>Policy based methods: PPO, DDP, A2C</li>
    <li>Not value based-methods: DQN</li>
    <li> Not actor critic mehods: Involves value-based methods</li>
    <li>Exploration strategy: maybe epislon greedy or Thompson sampling</li>
    <li> The trade-off between explorationa and exploitation happens here.</li>
</ul>

<h3> Training Loop </h3>
Simulation area for agent to interact in environment
<ul>
    <li>Collect state, action and reward</li>
    <li>Update policy and/or network using gradient based optimization</li>
    <li>Apply techniques like eperience replay and target networks</li>
    <!-- <li></li> -->
</ul>

<h3>Evaluation</h3>
Assess performance of agent
<ul>
    <li>Out-of-sample testing</li>
    <li> Performance measurement</li>
    <li> Attribution analysis</li>
    <!-- <li></li> -->
</ul>


<h3> Hyperparameter tuning and optimization</h3>

<ul>
    <li>Fine tune using: grid search, random search and bayesian optimization</li>
    <!-- <li></li>
    <li></li>
    <li></li> -->
</ul>


<h3>Saving and Loading Model</h3>
Ensure reproduability
<ul>
    <li> Save model</li>
    <!-- <li></li>
    <li></li>
    <li></li> -->
</ul>


In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import yfinance as yf

from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import minimize

import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

import torch as th
from torch import nn, optim

from Data_Retriever import DataRetriever as DatRet
from Markowitz_PT import MarkowitzPT as MPT
from Menchero_OGA import MencheroOGA as MOGA

In [2]:
# Defining stock pool
# Defining stock pool
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["EQNR.OL", "AKRBP.OL", "SUBC.OL", "BWO.OL",]
ticker_df["Seafood (food)"] = ["ORK.OL", "MOWI.OL", "SALM.OL", "LSG.OL"]
# ticker_df["Materials"] = ["NHY.OL", "YAR.OL", "RECSI.OL", "BRG.OL"]
ticker_df["Technologies"] = ["TEL.OL", "NOD.OL", "ATEA.OL", "BOUV.OL"]
ticker_df["Financial"] = ["STB.OL", "DNB.OL", "GJF.OL", "AKER.OL"]
ticker_df["Shipping"] = ["WAWI.OL", "SNI.OL", "BELCO.OL", "ODF.OL"]
ticker_df

# Defining ESG scores for respective securities
esg_scores = [36.6, 35.3, 17.9, 18, 
              18, 21.2, 18.7, 20,
              19.8, 13.8, 18.1, 19, 
              17.2, 14, 17.2, 19.5, 
              19.7, 21.2, 26.8, 19.3]

In [3]:
# # Retrieve data from yf API: y-m-d
# data = DatRet(ticker_df, "2013-01-01", "2024-01-01")

# # In function below, set log=True to check for data availability
# data.retrieve_data()

In [4]:
# return_array =  [pd.Series(stock) for sector in data.raw_data for stock in sector]
# return_df = pd.DataFrame(return_array)
# return_tdf = return_df.T

In [5]:
# return_tdf.to_csv('rawdata.csv', index=False)

In [6]:
return_tdf = pd.read_csv("rawdata.csv")

<h1>---------------------------------------------</h1>
<h2> Environment </h2>

<h2> PPO agent </h2>

In [7]:
from sklearn.model_selection import train_test_split

from RL_Algorithm.ThesisEnvironment  import PortfolioEnvironment as PorEnv
from RL_Algorithm.ThesisModel import LSTMPolicy as PolMod
from RL_Algorithm.ThesisModel import train_agent, test_agent

In [8]:
stock_data = return_tdf
esg_data = esg_scores

In [9]:
stock_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,85.244313,46.728795,107.098425,41.923917,27.966927,35.632132,25.396925,8.423798,54.311984,14.700000,27.263030,3.616566,21.400454,40.555292,39.259701,122.070627,14.458430,106.623315,2.097369,14.980077
1,85.788422,47.280163,108.292831,42.485039,28.355040,35.529247,25.903738,8.551432,54.456177,15.700000,27.487417,3.616566,21.633913,40.896091,39.817093,124.057823,14.387555,108.008034,2.097369,14.674361
2,85.848863,47.721272,108.372455,42.805673,28.355040,35.460665,26.466870,8.615249,54.504244,15.600000,27.375228,3.602656,21.766207,40.867690,39.986733,124.341742,14.316679,108.469607,2.139316,14.674363
3,85.727951,47.445583,107.974319,44.008080,28.263720,35.357780,25.734808,8.551432,54.792625,15.500000,27.263035,3.602656,21.828463,40.896091,39.817093,125.193398,14.316679,108.008034,2.190585,14.857792
4,85.727958,47.252600,108.053950,44.088255,28.366457,35.323488,25.734801,8.583340,54.648437,15.500000,27.263033,3.602656,21.836245,40.839280,39.502040,124.341714,14.245806,107.084888,2.190585,14.980078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5517,314.970886,265.288025,143.938995,22.236166,74.526024,171.770065,525.170410,39.213051,108.742676,122.050003,123.193367,58.351883,85.720802,196.734421,171.522629,655.955078,75.579575,308.253479,15.877379,93.699516
5518,313.962774,264.585696,144.035542,22.236165,74.410956,171.961564,524.417745,39.213050,108.742676,122.300003,123.193367,58.351887,85.720798,196.642619,171.240534,655.955081,76.864943,308.749861,15.997533,93.699524
5519,310.554321,260.635345,143.407837,22.137600,75.063011,172.631790,529.121887,39.250919,108.881912,125.199997,123.003242,58.162739,85.893005,196.459015,172.462997,654.980408,75.236816,308.253479,15.808722,94.517860
5520,310.554309,259.845331,143.697565,22.098173,75.273975,173.206268,530.439035,39.099444,109.067552,124.550003,122.052685,58.351887,85.835609,196.918025,172.368965,653.031078,75.408195,310.735424,15.791556,94.517853


In [10]:
# Assuming your stock_data and esg_data are numpy arrays
stock_data_train, stock_data_test= train_test_split(
    stock_data, test_size=0.2, shuffle=False
)

# Create training and testing environments
train_env = PorEnv(stock_data_train, esg_data, max_steps=50)
test_env = PorEnv(stock_data_test, esg_data, max_steps=50)

# Train the model on the training environment
trained_model = train_agent(train_env, learning_rate=0.001, episodes=50)
 
# Test the agent
test_agent(test_env, trained_model)


Episode 0 - Total Reward: 299.27336174447777
Episode 10 - Total Reward: 299.27336174447777
Episode 20 - Total Reward: 299.27336174447777
Episode 30 - Total Reward: 299.27336174447777
Episode 40 - Total Reward: 299.27336174447777
Total Test Reward: 75.48069003158317
Weights saved to weights_per_step.csv


[array([0.04322678, 0.02807658, 0.05608567, 0.06285782, 0.03809652,
        0.04405745, 0.05090551, 0.05415247, 0.06072159, 0.04815535,
        0.03592904, 0.05479523, 0.04711314, 0.05019377, 0.03996834,
        0.05003751, 0.05852829, 0.05488984, 0.05698498, 0.06522417],
       dtype=float32),
 array([0.04306129, 0.02871447, 0.05719469, 0.06222595, 0.03775988,
        0.04511488, 0.05010311, 0.05344335, 0.06252476, 0.04890586,
        0.03583354, 0.05593552, 0.04607116, 0.04913913, 0.03975571,
        0.04896266, 0.05782593, 0.05359009, 0.05689818, 0.06693981],
       dtype=float32),
 array([0.044586  , 0.02786965, 0.05709153, 0.06205787, 0.03903768,
        0.04404671, 0.05121848, 0.05131062, 0.06037643, 0.04851263,
        0.03740946, 0.05715111, 0.04698205, 0.0495594 , 0.03971909,
        0.04936404, 0.05607158, 0.05565102, 0.0564656 , 0.06551905],
       dtype=float32),
 array([0.0427084 , 0.02830462, 0.05589046, 0.06158399, 0.0390364 ,
        0.04340155, 0.05196076, 0.05639444, 

In [11]:
my_df = pd.read_csv("weights_per_step.csv")